In [35]:
import os

import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pyproj as proj
from datetime import datetime
import pandas as pd
from netCDF4 import Dataset  # pylint:disable=no-name-in-module
from shapely.geometry import Point
import altair as alt
import vl_convert as vlc
import json
import topojson
from IPython.display import Image

alt.data_transformers.enable("vegafusion")
sf_type = 1
marker_size = 1

# Antarctica

In [36]:
ais_basins = gpd.read_file("aux_files/IMBIE_AIS_Basins/ANT_Basins_IMBIE2_v1.6.shp")
ais_basins = ais_basins.reset_index().rename(columns={"index": "basin_id"}).to_crs("epsg:4326")
ais_basins = ais_basins.drop(labels=np.where(ais_basins["Regions"] == "Islands")[0], axis=0)
ais_basins["basin_id"] = (ais_basins["basin_id"]).astype(str)
ais_basins.head(20)

,basin_id,Regions,Subregion,geometry
1,1,West,H-Hp,"POLYGON ((-90.24588 -74.02074, -90.53000 -73.9..."
2,2,West,F-G,"POLYGON ((-144.36800 -75.62323, -144.36771 -75..."
3,3,East,E-Ep,"POLYGON ((-151.52291 -85.37243, -151.30170 -85..."
4,4,East,D-Dp,"POLYGON ((144.62895 -67.18320, 144.62074 -67.1..."
5,5,East,Cp-D,"POLYGON ((104.60697 -68.21552, 104.62590 -68.1..."
6,6,East,B-C,"POLYGON ((75.87282 -80.84156, 75.23614 -80.998..."
7,7,East,A-Ap,"POLYGON ((-5.68899 -72.94988, -5.77515 -72.721..."
8,8,East,Jpp-K,"POLYGON ((-47.77402 -81.92722, -47.66685 -81.9..."
9,9,West,G-H,"POLYGON ((-102.82001 -72.70919, -102.86595 -72..."
10,10,East,Dp-E,"POLYGON ((167.40822 -73.51534, 167.39224 -73.5..."


In [37]:
ais_basins.to_file("aux_files/ais_basins.geojson", driver="GeoJSON")

In [21]:
with open("aux_files/ais_basins.json", "w") as fp:
    fp.write(topojson.Topology(ais_basins).to_json())
    

In [39]:
del ais_basins

In [40]:
ais_basins = gpd.GeoDataFrame.from_file("aux_files/ais_basins.geojson")
ais_basins

,basin_id,Regions,Subregion,geometry
0,1,West,H-Hp,"POLYGON ((-90.24588 -74.02074, -90.53000 -73.9..."
1,2,West,F-G,"POLYGON ((-144.36800 -75.62323, -144.36771 -75..."
2,3,East,E-Ep,"POLYGON ((-151.52291 -85.37243, -151.30170 -85..."
3,4,East,D-Dp,"POLYGON ((144.62895 -67.18320, 144.62074 -67.1..."
4,5,East,Cp-D,"POLYGON ((104.60697 -68.21552, 104.62590 -68.1..."
5,6,East,B-C,"POLYGON ((75.87282 -80.84156, 75.23614 -80.998..."
6,7,East,A-Ap,"POLYGON ((-5.68899 -72.94988, -5.77515 -72.721..."
7,8,East,Jpp-K,"POLYGON ((-47.77402 -81.92722, -47.66685 -81.9..."
8,9,West,G-H,"POLYGON ((-102.82001 -72.70919, -102.86595 -72..."
9,10,East,Dp-E,"POLYGON ((167.40822 -73.51534, 167.39224 -73.5..."


In [41]:
timedata_ais = pd.read_csv(
    "./processed_files/time_series_data_AIS.csv"
)

# max_v_ais = np.round(timedata_ais["Raw SEC"].abs().max(axis=None) + 0.05, 1)
# timedata_ais["basin"] = timedata_ais["basin"].astype(str)
# timedata_ais = pd.merge(
#     timedata_ais,
#     ais_basins[["basin_id", "Subregion"]],
#     how="outer",
#     left_on="basin",
#     right_on="basin_id",
# ).drop(columns=["basin_id"])
# basins = timedata_ais["basin"]
# timedata_ais.loc[timedata_ais["basin"] == "all", "Subregion"] = "All"
# timedata_ais = timedata_ais.drop(columns=["basin"])
# timedata_ais

In [42]:


click_basin_ais = alt.selection_point(fields=["Subregion"])

map_ais = (
    alt.Chart(
        ais_basins,
    )
    .mark_geoshape()
    .encode(
        color=alt.Color("Subregion:N").legend(None),
        opacity=alt.condition(click_basin_ais, alt.value(1), alt.value(0.2)),
        tooltip=["Subregion:N", "Regions:N"],
    )
    .project(type="stereographic", center=[-90, -180])
    .properties(width=500, height=500)
)

jchart1 = alt.JupyterChart(map_ais.add_params(click_basin_ais))
jchart1

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_6_store'}…

In [43]:
sec_ais = (
    alt.Chart(timedata_ais, title="Mean SEC per Basin")
    .mark_line()
    .encode(
        alt.X("midpoint:T", title="Time Period"),
        alt.Y(
            "Smooth SEC:Q",
            title="Mean elevation change (m/year)",
        ),
        opacity=alt.condition(click_basin_ais, alt.value(1), alt.value(0.05)),
        color=alt.Color("Subregion:N"),
        tooltip=["Raw SEC", "period", "Subregion"],
    )
    .properties(width=800, height=200)
)

dh_ais = (
    alt.Chart(timedata_ais, title="Total Mean SEC since 1991 per basin")
    .mark_line()
    .encode(
        alt.X("midpoint:T", title="Time Period"),
        alt.Y(
            "dH:Q",
            title="Total elevation change (m)",
        ),
        opacity=alt.condition(click_basin_ais, alt.value(1), alt.value(0.05)),
        color=alt.Color("Subregion:N"),
        tooltip=["dH", "period", "Subregion"],
    )
    .properties(width=800, height=200)
)

zero = pd.DataFrame([{"zero": 0.0}])
zero_rule = (
    alt.Chart(zero).mark_rule(color="black", strokeDash=[1]).encode(y="zero:Q", size=alt.value(1))
)

line_chart_ais = (sec_ais + zero_rule) & (dh_ais + zero_rule)

jchart2 = alt.JupyterChart(line_chart_ais.add_params(click_basin_ais))
jchart2

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_6_store'}…

In [44]:
jchart3 = alt.JupyterChart((line_chart_ais | map_ais).add_params(click_basin_ais))
jchart3

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_6_store'}…

In [45]:
with open("ais_spec.json","w") as fp:
    json.dump(jchart3.spec, fp)

# Greenland

In [46]:
crs_new = ccrs.NorthPolarStereo(central_longitude=-40)
gis_basins = gpd.read_file(
    "aux_files/IMBIE_GIS_Basins/Greenland_Basins_PS_v1.4.2.shp"
).to_crs("EPSG:4326")
gis_basins = gis_basins.reset_index().rename(
    columns={"index": "basin_id", "SUBREGION1": "Subregion"}
)
gis_basins["basin_id"] = (gis_basins["basin_id"] + 1).astype(str)
gis_basins.head(10)

,basin_id,Subregion,NAME,GL_TYPE,geometry
0,1,CE,None,None,"POLYGON ((-32.84621 68.55410, -32.84621 68.554..."
1,2,CW,None,None,"POLYGON ((-51.20762 71.61893, -51.20951 71.618..."
2,3,NO,None,None,"POLYGON ((-71.46118 78.14071, -71.65543 78.133..."
3,4,NE,None,None,"POLYGON ((-22.65215 79.51328, -22.65259 79.511..."
4,5,NW,None,None,"POLYGON ((-52.26019 71.88370, -52.27234 71.886..."
5,6,SE,None,None,"POLYGON ((-39.14148 65.77665, -39.14072 65.779..."
6,7,SW,None,None,"POLYGON ((-50.24135 68.22278, -50.20214 68.234..."


In [47]:
with open("aux_files/gris_basins.json", "w") as fp:
    fp.write(gis_basins.to_json())

In [48]:
click_basin_gis = alt.selection_point(fields=["Subregion"])

map_gis = (
    alt.Chart(
        gis_basins.to_crs(epsg="4326"),
    )
    .mark_geoshape()
    .encode(
        color=alt.Color(
            "Subregion:N",
            sort=["All", "NO", "NE", "NW", "CE", "CW", "SE", "SW"],
        ).legend(orient="right"),
        opacity=alt.condition(click_basin_gis, alt.value(1), alt.value(0.2)),
        tooltip=["basin_id:N", "Subregion:N"],
    )
    .project("stereographic", rotate=[45, -90])
    .properties(width=400, height=400)
)

jchart4 = alt.JupyterChart(map_gis.add_params(click_basin_gis), debounce_wait=1000)
jchart4

JupyterChart(debounce_wait=1000.0, spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'n…

In [49]:
jchart4.spec.pop("data")
with open("spec.json", "w")as fp:
    json.dump(jchart4.spec, fp, sort_keys=True)

In [50]:
timedata_gis = pd.read_csv(
    "./processed_files/time_series_data_GIS.csv"
)

timedata_gis

,period,midpoint,basin,Raw SEC,Smooth SEC,dH
0,1991/07 - 1996/07,1994-01-28 12:02:43.476562,1,-0.008,-0.008,0.000000
1,1991/07 - 1996/07,1994-01-28 12:02:43.476562,2,-0.124,-0.124,0.000000
2,1991/07 - 1996/07,1994-01-28 12:02:43.476562,3,0.032,0.032,0.000000
3,1991/07 - 1996/07,1994-01-28 12:02:43.476562,4,0.035,0.034,0.000000
4,1991/07 - 1996/07,1994-01-28 12:02:43.476562,5,-0.049,-0.049,0.000000
...,...,...,...,...,...,...
2611,2018/09 - 2023/09,2021-03-31 01:17:45.234375,4,-0.002,-0.002,4.547018
2612,2018/09 - 2023/09,2021-03-31 01:17:45.234375,5,-0.194,-0.194,-35.727822
2613,2018/09 - 2023/09,2021-03-31 01:17:45.234375,6,-0.114,-0.114,-22.271594
2614,2018/09 - 2023/09,2021-03-31 01:17:45.234375,7,-0.130,-0.130,-18.662165


In [51]:
line_gis = (
    alt.Chart(timedata_gis, title="Mean SEC per Basin")
    .mark_line()
    .encode(
        alt.X(
            "midpoint:T",
            axis=alt.Axis(tickMinStep=2),
            title="Time Period",
        ),
        alt.Y(
            "Smooth SEC:Q",
            scale=alt.Scale(domain=(-max_v_gis, max_v_gis)),
            title="Mean elevation change (m/year)",
        ),
        opacity=alt.condition(click_basin_gis, alt.value(1), alt.value(0.05)),
        color=alt.Color(
            "Subregion:N",
            sort=["All", "Outside basin", "NO", "NE", "NW", "CE", "CW", "SE", "SW"],
        ).legend(orient="right"),
        tooltip=["Smooth SEC", "period", "Subregion"],
    )
    .properties(width=900, height=200)
)

dh_gis = (
    alt.Chart(timedata_gis, title="Total SEC per basin")
    .mark_line()
    .encode(
        alt.X("midpoint:T", title="Time Period"),
        alt.Y(
            "dH:Q",
            title="Total elevation change (m)",
        ),
        opacity=alt.condition(click_basin_gis, alt.value(1), alt.value(0.05)),
        color=alt.Color("Subregion:N"),
        tooltip=["dH", "period", "Subregion"],
    )
    .properties(width=900, height=200)
)


zero = pd.DataFrame([{"zero": 0.0}])
zero_rule = (
    alt.Chart(zero)
    .mark_rule(color="black", strokeDash=[1])
    .encode(y="zero:Q", size=alt.value(1))
)

line_chart_gis = (line_gis + zero_rule) & (dh_gis + zero_rule)

jchart4 = alt.JupyterChart(line_chart_gis.add_params(click_basin_gis))
jchart4

NameError: name 'max_v_gis' is not defined

In [ ]:
chart = (line_chart_gis | map_gis).add_params(click_basin_gis)


jchart = alt.JupyterChart(chart)
jchart

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_8_store'}…

In [ ]:
with open("gris_spec.json","w") as fp:
    json.dump(jchart.spec, fp)